## Online Learning Perceptro in Python
파이썬으로 표준 라이브러리를 사용해서 위의 퍼셉트론 알고리즘을 구현했기 때문에 스크립트가 PyPy에서 실행되고 3-4배의 속도향상이 있다. 여기에 사용된 알고리즘은 Kaggle 포럼에서 처음 발견 된 tinrtgu의 온라인 로지스틱 회귀 스크립트에서 큰 영감을 얻었다고 한다.

다음 경진대회에서 Vowpal Wabbit을 통한 해시트릭을 사용하였고 코드가 공개되어 있다.

Display Advertising Challenge - Kaggle, Beat the benchmark with less then 200MB of memory.
코드 : https://kaggle2.blob.core.windows.net/forum-message-attachments/53646/1539/fast_solution.py

## 해싱 트릭
벡터화 해싱트릭은 Vowpal Wabbit(John Langford)에서 시작되었다. 이 트릭은 퍼셉트론으로 들어오는 연결 수를 고정 된 크기로 설정한다. 고정 된 크기보다 낮은 숫자로 모든 원시 피처를 해싱한다. Vowpal Wabbit은 모든 데이터를 메모리로 읽어들이지 않고 모델을 훈련시킬 수 있는 빠른 머신러닝 라이브러리다.

In [ ]:
sample = 'This movie sucks'
fixed_size = 1024

print(sample.split())

features = [(hash(f)%fixed_size, 1) for f in sample.split()]

# list of tuples in form (feature_index, feature_value)
print(features)

## Data extract

In [ ]:
!unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip

In [ ]:
PATH = '/kaggle/working/'

## 프로그레시브 검증 손실
한 번에 하나씩 표본을 학습하면 점신적으로 train loss가 된다. 모델이 타겟을 보지않고 첫 샘플을 보고 예측을 한다. 그런 다음 예측을 대상 레이블과 비교하여 오류율을 계산한다. 오류율이 낮으면 좋은 모델에 가깝다.

In [ ]:
import re
import random
from math import exp, log
from datetime import datetime
from operator import itemgetter # 키가 아닌 값으로 max, min 값을 구할 때 사용

In [ ]:
def clean(s):
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

In [ ]:
def get_data_tsv(loc_dataset, opts):
    for e, line in enumerate(open(loc_dataset, 'rb')):
        if e > 0:
            r = line.decode('utf-8').strip().split('\t')
            id = r[0]
            
            if opts['clean']:
                try:
                    r[2] = clean(r[2])
                except:
                    r[1] = clean(r[1])
            
            if len(r) == 3:
                features = [(hash(f)%opts['D'], 1) for f in r[2].split()]
                label = int(r[1])
            else:
                features = [(hash(f)%opts['D'], 1) for f in r[1].split()]
                label = 1
            
            if opts['2grams']:
                for i in range(len(features)-1):
                    features.append(
                        (hash(str(features[i][0])+str(features[i+1][0]))%opts['D'], 1))
            yield label, id, features

In [ ]:
def dot_product(features, weights):
    dotp = 0
    for f in features:
        dotp += weights[f[0]]*f[1]
    return dotp

In [ ]:
def train_tron(loc_dataset, opts):
    start = datetime.now()
    print('\nPass\t\tErrors\t\tAverage\t\tNr. Samples\tSince Start')
    
    # 가중치 초기화
    if opts['random_init']:
        random.seed(3003)
        weight = [random.random()] * opts['D']
    else:
        weights = [0.] * opts['D']
    
    # Running training passes
    # 학습 실행
    for pass_nr in range(opts['n_passes']):
        error_counter = 0
        for e, (label, id, features) in enumerate( \
            get_data_tsv(loc_dataset, opts)):
            
            # 퍼셉트론은 지도학습 분류기의 일종
            # 이전 값에 대한 학습으로 예측
            # 내적(dotproduct) 값이 임계 값보다 높거나 낮은지에 따라
            # 초과하면 1을 예측하고 미만이면 0을 예측한다.
            dp = dot_product(features, weights) > 0.5
            
            # 다음 perceptron은 샘플의 레이블을 본다.
            # 실제 레이블 데이터에서 위 퍼셉트론으로 구한 dp 값을 빼준다.
            # 예측이 정확하다면 error 값은 0이며, 가중치만 남겨둔다.
            # 예측이 틀린 경우 error값은 1 또는 -1이고 다음과 같이 가중치를 업데이트 한다.
            # weights[feature_index] += learning_rate * error * feature_value
            
            error = label - dp
            
            # 예측이 틀린 경우 퍼셉트론은 다음과 같이 가중치를 업데이트 한다.
            if error != 0:
                error_counter += 1
                # updating the weights
                for index, value in features:
                    weights[index] += opts['learning_rate'] * error * log(1.+value)
        
        # Reporting stuff
        print('%s\t\t%s\t\t%s\t\t%s\t\t%s' % (\
                pass_nr+1,
                error_counter,
                round(1 - error_counter / float(e+1), 5),
                e+1, datetime.now()-start))
        
        # Oh heh, we have overfit :)
        if error_counter == 0 or error_counter < opts['errors_satisfied']:
            print('%s erros found during training, halting' % error_counter)
            break
    return weights

In [ ]:
def test_tron(loc_dataset,weights,opts):
    """
        output:
                preds: list, a list with
                [id,prediction,dotproduct,0-1normalized dotproduct]
    """
    start = datetime.now()
    print("\nTesting online\nErrors\t\tAverage\t\tNr. Samples\tSince Start")
    preds = []
    error_counter = 0
    for e, (label, id, features) in enumerate( \
        get_data_tsv(loc_dataset,opts) ):

        dotp = dot_product(features, weights)
        # 내적이 0.5보다 크다면 긍정으로 예측한다.
        dp = dotp > 0.5
        if dp > 0.5: # we predict positive class
            preds.append( [id, 1, dotp ] )
        else:
            preds.append( [id, 0, dotp ] )
        
        # get_data_tsv에서 테스트 데이터의 레이블을 1로 초기화 해주었음
        if label - dp != 0:
            error_counter += 1

    print("%s\t\t%s\t\t%s\t\t%s" % (
        error_counter,
        round(1 - error_counter /float(e+1),5),
        e+1,
        datetime.now()-start))

    # normalizing dotproducts between 0 and 1 
    # 내적을 구해 0과 1로 일반화 한다.
    # TODO: proper probability (bounded sigmoid?), 
    # online normalization
    max_dotp = max(preds,key=itemgetter(2))[2]
    min_dotp = min(preds,key=itemgetter(2))[2]
    for p in preds:
        # appending normalized to predictions
        # 정규화 된 값을 마지막에 추가해 준다.
        # (피처와 가중치에 대한 내적값 - 최소 내적값) / 최대 내적값 - 최소 내적값
        # 이 값이 캐글에서 0.95의 AUC를 얻을 수 있는 값이다.
        p.append((p[2]-min_dotp)/float(max_dotp-min_dotp)) 
        
    #Reporting stuff
    print("Done testing in %s"%str(datetime.now()-start))
    return preds

In [ ]:
#Setting options
opts = {}
opts["D"] = 2 ** 25
opts["learning_rate"] = 0.1
opts["n_passes"] = 80 # Maximum number of passes to run before halting
opts["errors_satisfied"] = 0 # Halt when training errors < errors_satisfied
opts["random_init"] = False # set random weights, else set all 0
opts["clean"] = True # clean the text a little
opts["2grams"] = True # add 2grams

#training and saving model into weights
%time 
weights = train_tron(PATH+"labeledTrainData.tsv",opts)

In [ ]:
# testing and saving predictions into preds
%time 
preds = test_tron(PATH+"testData.tsv",weights,opts)

In [ ]:
preds[:10]

In [ ]:
# 캐글 점수 제출을 위한 서브미션 파일을 작성한다.
with open("submit_perceptron.csv","wb") as outfile:
    outfile.write('"id","sentiment"\n'.encode('utf-8'))
    for p in sorted(preds):
        outfile.write("{},{}\n".format(p[0],p[3]).encode('utf-8'))